# `GiRaFFE_NRPy`: Numerical Methods

## Authors: Patrick Nelson &

In this module, we will introduce the numerical methods with which we will solve the GRFFE equations.

The contributions from flux terms to the right-hand sides of the conservation equations is found using PPM reconstruction and HLLE approximate Riemann solvers. The other right-hand side terms, as well as the magnetic field from the vector potential, are calculated with sipmle